In [2]:
from classes.SAMMed2DClass import SAMMed2DInferer
from utils.base_classes import Points
import utils.analysis as anUt
import utils.prompt as prUt
from utils.image import read_im_gt
from utils.interactivity import iterate_2d

import numpy as np

# Obtain model
device = 'cuda'
sammed2d_checkpoint_path = "/home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth"
sammed2d_inferer = SAMMed2DInferer(sammed2d_checkpoint_path, device)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


True
*******load /home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth


In [3]:
# Obtain image, gt
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = read_im_gt(img_path, gt_path, 3)


In [3]:
# Experiment: n randomly sampled points from foreground
seed = 11121
# n_clicks = 1
n_clicks = 5
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_clicks, seed = seed)
segmentation = sammed2d_inferer.predict(img, point_prompt)
anUt.compute_dice(segmentation, gt)

Performing inference on slices: 100%|██████████| 45/45 [00:02<00:00, 19.79it/s]


0.9143276197755548

In [25]:
# Experiment: 2d bounding box per slice with foreground
box_prompt = prUt.get_minimal_boxes_row_major(gt, 0, 0)
segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 174.58it/s]


0.9480396762401525

In [26]:
# Experiment: get 3d bbox and slice it to feed it in 2d axially
box_prompt = prUt.get_bbox3d_sliced(gt)

segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 176.60it/s]


0.7636174284813798

In [27]:
# Experiment: line interpolation
n_slices = 5
interpolation = 'linear'
point_prompt = prUt.line_interpolation(gt, n_slices, interpolation)

segmentation = sammed2d_inferer.predict(img, point_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 147.57it/s]


0.912622782583199

In [28]:
# Experiment: box interpolation
n_boxes = 5

seed_boxes = prUt.get_seed_boxes(gt, n_boxes)
box_prompt = prUt.box_interpolation(seed_boxes)
segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 167.36it/s]


0.9463972842750766

In [10]:
# Experiment: Point propagation
seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks)
print(anUt.compute_dice(gt,segmentation))


Propagating up: 100%|██████████| 12/12 [00:00<00:00, 84.53it/s]


0.6715563146789912


In [7]:
# Iteratively improve previous segmentation (from point propagation)
seed = 1
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks)


initial_prompt = prompt
condition = 'dof'
dof_bound = 90
perf_bound = 0.85
dice_scores, dof, segmentations, prompts = iterate_2d(sammed2d_inferer, img, gt, segmentation, initial_prompt, sammed2d_inferer.pass_prev_prompts,
                                                      scribble_length = 0.6, contour_distance = 3, disk_size_range= (0,3),
                                                      init_dof = 5, perf_bound = 0.85, dof_bound = dof_bound, seed = seed, detailed = True)

Propagating up: 100%|██████████| 22/22 [00:00<00:00, 96.12it/s] 


In [11]:
# Experiment: Box propagation

seed_box = prUt.get_seed_box(gt)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.box_propagation(sammed2d_inferer, img, seed_box, slices_to_infer)

print(anUt.compute_dice(gt,segmentation))

Propagating up: 100%|██████████| 12/12 [00:00<00:00, 66.52it/s]


0.5264982538428203
